# MorseGraph Example 3: ODE-Based Dynamics

This notebook demonstrates how to compute a Morse graph for a dynamical system defined by an Ordinary Differential Equation (ODE).

We will use the `BoxMapODE` dynamics class, which uses a numerical integrator (`scipy.integrate.solve_ivp`) to approximate the flow of the system over a time horizon `tau`. As an example, we will analyze a simple bistable system with two stable fixed points and a saddle point.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import the necessary components from the MorseGraph library
from morsegraph.grids import UniformGrid
from morsegraph.dynamics import BoxMapODE
from morsegraph.core import Model
from morsegraph.analysis import compute_morse_graph
from morsegraph.plot import plot_morse_graph, plot_morse_sets

## 1. Define the ODE System

We'll use a simple 2D bistable system defined by the equations:
- `dx/dt = x - x^3`
- `dy/dt = -y`

This system has stable fixed points (sinks) at `(1, 0)` and `(-1, 0)`, and an unstable fixed point (a saddle) at `(0, 0)`. Let's define the function and visualize the vector field to get a feel for the dynamics.

In [ ]:
def bistable_ode(t, x):
    """
    A simple bistable ODE system.
    dx/dt = x - x[0]**3
    dy/dt = -y
    """
    dxdt = x[0] - x[0]**3
    dydt = -x[1]
    return np.array([dxdt, dydt])

# Visualize the vector field
x, y = np.meshgrid(np.linspace(-2, 2, 20), np.linspace(-2, 2, 20))
u, v = bistable_ode(0, np.array([x, y]))

plt.figure(figsize=(8, 8))
plt.quiver(x, y, u, v, color='blue')
plt.title("Vector Field of the Bistable ODE")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

## 2. Set up the Morse Graph Computation

Now, we define the grid and the `BoxMapODE` dynamics model. A key parameter here is `tau`, the integration time. It should be large enough to capture the movement between regions but not so large that the entire domain collapses to the attractors in one step.

In [ ]:
# Define the grid parameters
subdivisions = 15
domain = np.array([[-2.0, 2.0], [-2.0, 2.0]])

# 1. Create the dynamics object for our ODE
# We choose tau=0.5 as the integration time.
dynamics = BoxMapODE(
    ode_func=bistable_ode,
    dimension=2,
    tau=0.5,
    bloat_factor=0.1
)

# 2. Create the grid
grid = UniformGrid(subdivisions, domain)

# 3. Create the model
model = Model(grid, dynamics)

# 4. Compute the map graph
print("Computing map graph...")
map_graph = model.compute_map_graph()
print("Map graph computed.")

# 5. Compute the Morse graph
morse_graph = compute_morse_graph(map_graph)

## 3. Visualize the Results

We expect the Morse graph to show three primary Morse sets: two sinks (the attractors) and one source/saddle. The graph should show that the saddle flows towards the two sinks.

In [ ]:
# Plot the Morse graph
fig, ax = plt.subplots()
plot_morse_graph(morse_graph, ax=ax)
ax.set_title("Morse Graph for Bistable ODE")
plt.show()

# Plot the Morse sets on the grid
fig, ax = plt.subplots(figsize=(8, 8))
plot_morse_sets(morse_graph, grid, ax=ax)
ax.set_title("Morse Sets for Bistable ODE")
# Overlay the vector field for context
plt.quiver(x, y, u, v, color='blue', alpha=0.3)
plt.show()